## Section 1️⃣: Import Required Modules

Import all necessary libraries and custom modules from the `src/` folder.

In [ ]:
# ============================================================
# SECTION 1: IMPORT MODULES
# ============================================================

print("\n" + "="*70)
print("IMPORTING MODULES FROM SRC PACKAGE")
print("="*70)

# ============================================================
# 1.1: Standard Libraries
# ============================================================

import os
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, auc,
    precision_recall_curve, average_precision_score
)
from sklearn.preprocessing import label_binarize
from tqdm import tqdm

print("\n✅ Standard libraries imported")

# ============================================================
# 1.2: Custom Modules from src/
# ============================================================

# These are the key modules we'll use throughout the notebook
from src.models import EnhancedJointECGModel, ECGEncoder, GCNEncoder, ClassConditionedDecoder, SimpleClassifier
from src.data_loader import get_dataloaders, load_ecg_data, get_class_weights, ECGDataset
from src.graph_utils import build_knn_graph, compute_edge_scores

print("✅ Custom modules imported from src/")
print("   - src.models: Neural network architectures")
print("   - src.data_loader: Data loading utilities")
print("   - src.graph_utils: Graph construction utilities")

# ============================================================
# 1.3: Set Up Visualization
# ============================================================

plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("\n✅ Visualization setup complete")

# ============================================================
# 1.4: Set Random Seeds for Reproducibility
# ============================================================

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

print("\n✅ Random seeds set for reproducibility")

# ============================================================
# 1.5: Check GPU Availability
# ============================================================

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🚀 Using device: {device}")

if device.type == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print(f"   Current GPU memory: {torch.cuda.memory_allocated() / 1e9:.1f} GB / {torch.cuda.memory_reserved() / 1e9:.1f} GB")

print(f"\n{'='*70}")

## Section 2️⃣: Configuration and Data Loading

Set up configuration, load ECG data using `src.data_loader`, and create train/validation splits.

In [ ]:
# ============================================================
# SECTION 2: CONFIGURATION AND DATA LOADING
# ============================================================

print("\n" + "="*70)
print("CONFIGURATION AND DATA LOADING")
print("="*70)

# ============================================================
# 2.1: Configuration
# ============================================================

CONFIG = {
    # Data paths
    'train_path': 'examples/sample_ecgs.csv',      # ← Update if you have full dataset
    'test_path': None,                             # ← Optional
    
    # Model parameters
    'input_length': 187,
    'latent_dim': 128,
    'num_classes': 5,
    
    # Training parameters
    'num_epochs': 50,                              # ← Reduce for demo, use 200+ for real training
    'batch_size': 32,                              # ← Small batch for demo
    'learning_rate': 0.001,
    'k_neighbors': 5,
    'device': device,
    
    # Loss weights
    'lambda_class': 0.5,
    'lambda_graph': 0.1,
}

# Class names
CLASS_NAMES = ['Normal (N)', 'Supraventricular (S)', 'Ventricular (V)', 'Fusion (F)', 'Unknown (Q)']

print("\n✅ Configuration loaded:")
for key, value in CONFIG.items():
    if key != 'device':
        print(f"   {key:20s}: {value}")

print(f"\n   device              : {CONFIG['device']}")

# ============================================================
# 2.2: Load Data Using src.data_loader Module
# ============================================================

print(f"\n📊 Loading ECG data from {CONFIG['train_path']}...")

# Option 1: Use get_dataloaders for automatic train/val split
train_loader, val_loader = get_dataloaders(
    train_path=CONFIG['train_path'],
    batch_size=CONFIG['batch_size'],
    val_split=0.15,
    normalize=True,
    num_workers=0  # Set to 0 for Jupyter notebooks
)

# Option 2: Manual loading for more control (commented out)
# waveforms, labels = load_ecg_data(CONFIG['train_path'], normalize=True)
# dataset = ECGDataset(waveforms, labels)

print("✅ Data loaded successfully")
print(f"   Train batches: {len(train_loader)}")
print(f"   Validation batches: {len(val_loader)}")
print(f"   Batch size: {CONFIG['batch_size']}")

# Get sample batch to verify shapes
sample_batch_x, sample_batch_y = next(iter(train_loader))
print(f"\n📐 Data shapes:")
print(f"   Waveforms: {sample_batch_x.shape}  (batch_size, channels, time_steps)")
print(f"   Labels: {sample_batch_y.shape}")

print(f"\n{'='*70}")

## Section 3️⃣: Initialize Model

Create the Class-Conditional Graph Autoencoder, optimizer, and scheduler using modules from `src.models`.

In [ ]:
# ============================================================
# SECTION 3: INITIALIZE MODEL
# ============================================================

print("\n" + "="*70)
print("INITIALIZING MODEL")
print("="*70)

# ============================================================
# 3.1: Create Model Instance
# ============================================================

print("\n🧠 Creating EnhancedJointECGModel...")

model = EnhancedJointECGModel(
    input_length=CONFIG['input_length'],
    latent_dim=CONFIG['latent_dim'],
    num_classes=CONFIG['num_classes']
).to(CONFIG['device'])

# Count parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"\n✅ Model created successfully")
print(f"   Total parameters: {num_params:,}")

# Show model summary
print(f"\n📐 Model Components:")
print(f"   ├─ ECGEncoder: CNN with 3 conv layers")
print(f"   ├─ GCNEncoder: 2-layer Graph Convolutional Network")
print(f"   ├─ ClassConditionedDecoder: Deconvolutional network")
print(f"   └─ SimpleClassifier: 2-layer dense network")

# ============================================================
# 3.2: Set Up Optimizer
# ============================================================

print("\n🔧 Setting up optimizer...")

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=CONFIG['learning_rate'],
    weight_decay=1e-5
)

print(f"✅ Optimizer configured: Adam")
print(f"   Learning rate: {CONFIG['learning_rate']}")
print(f"   Weight decay: 1e-5")

# ============================================================
# 3.3: Learning Rate Scheduler
# ============================================================

print("\n⏱️  Setting up learning rate scheduler...")

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=True
)

print(f"✅ Scheduler configured: ReduceLROnPlateau")
print(f"   Reduces LR by 50% if no improvement for 5 epochs")

print(f"\n{'='*70}")

## Section 4️⃣: Build k-NN Graphs

Use `src.graph_utils.build_knn_graph()` to construct graphs from embeddings.

In [ ]:
# ============================================================
# SECTION 4: TEST GRAPH CONSTRUCTION
# ============================================================

print("\n" + "="*70)
print("TESTING GRAPH CONSTRUCTION")
print("="*70)

# ============================================================
# 4.1: Create Sample Data and Build Graph
# ============================================================

print("\n🔗 Building k-NN graph for a sample batch...")

# Get a sample batch
sample_x, sample_y = next(iter(train_loader))
sample_x = sample_x.to(CONFIG['device'])

# Extract features using CNN encoder
with torch.no_grad():
    sample_features = model.ecg_encoder(sample_x)

print(f"\n   CNN features shape: {sample_features.shape}")
print(f"   (batch_size={CONFIG['batch_size']}, feature_dim={CONFIG['latent_dim']})")

# Build k-NN graph
edge_index = build_knn_graph(
    sample_features,
    k=CONFIG['k_neighbors'],
    metric='cosine'
)

print(f"\n✅ Graph constructed successfully")
print(f"   Edge index shape: {edge_index.shape}")
print(f"   Number of edges: {edge_index.shape[1]}")
print(f"   Average edges per node: {edge_index.shape[1] / CONFIG['batch_size']:.1f}")
print(f"   (Expected: ~{2 * CONFIG['k_neighbors']} for bidirectional graph)")

# ============================================================
# 4.2: Verify Graph Properties
# ============================================================

print(f"\n🔍 Graph Properties:")

# Check connectivity
connected_nodes = torch.unique(edge_index)
print(f"   Connected nodes: {len(connected_nodes)} / {CONFIG['batch_size']}")

# Check degree distribution
from torch_geometric.utils import degree
deg = degree(edge_index[0], num_nodes=CONFIG['batch_size'])
print(f"   Node degree - Min: {deg.min().item():.1f}, Max: {deg.max().item():.1f}, Avg: {deg.mean().item():.1f}")

print(f"\n{'='*70}")

## Section 5️⃣: Training Loop

Execute the training loop with monitoring and validation.

In [ ]:
# ============================================================
# SECTION 5: TRAINING LOOP
# ============================================================

print("\n" + "="*70)
print("TRAINING CLASS-CONDITIONAL GAE")
print("="*70)

# ============================================================
# 5.1: Initialize Training Storage
# ============================================================

history = {
    'train_loss': [],
    'train_recon_loss': [],
    'train_class_loss': [],
    'train_graph_loss': [],
    'val_loss': [],
    'val_acc': []
}

best_val_acc = 0
best_epoch = 0

# ============================================================
# 5.2: Main Training Loop
# ============================================================

print(f"\n🚀 Starting training for {CONFIG['num_epochs']} epochs...\n")

for epoch in range(1, CONFIG['num_epochs'] + 1):
    # ============================================================
    # Training Phase
    # ============================================================
    
    model.train()
    epoch_losses = {'total': 0, 'recon': 0, 'class': 0, 'graph': 0}
    num_batches = 0
    
    for batch_x, batch_y in train_loader:
        batch_x = batch_x.to(CONFIG['device'])
        batch_y = batch_y.to(CONFIG['device'])
        
        # Build k-NN graph
        with torch.no_grad():
            features = model.ecg_encoder(batch_x)
        edge_index = build_knn_graph(features, k=CONFIG['k_neighbors'])
        
        # Forward pass (training mode: use TRUE labels)
        optimizer.zero_grad()
        z, reconstructed, logits = model(batch_x, edge_index, batch_y)
        
        # Compute loss
        loss, loss_dict = model.compute_loss(
            batch_x, reconstructed, logits, batch_y, edge_index, z
        )
        
        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        # Accumulate losses
        epoch_losses['total'] += loss.item()
        epoch_losses['recon'] += loss_dict['recon']
        epoch_losses['class'] += loss_dict['class']
        epoch_losses['graph'] += loss_dict['graph']
        num_batches += 1
    
    # Average losses
    for key in epoch_losses:
        epoch_losses[key] /= num_batches
    
    history['train_loss'].append(epoch_losses['total'])
    history['train_recon_loss'].append(epoch_losses['recon'])
    history['train_class_loss'].append(epoch_losses['class'])
    history['train_graph_loss'].append(epoch_losses['graph'])
    
    # ============================================================
    # Validation Phase
    # ============================================================
    
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for val_x, val_y in val_loader:
            val_x = val_x.to(CONFIG['device'])
            val_y = val_y.to(CONFIG['device'])
            
            # Build graph
            val_features = model.ecg_encoder(val_x)
            val_edge_index = build_knn_graph(val_features, k=CONFIG['k_neighbors'])
            
            # Inference mode: use PREDICTED classes
            z_val, recon_val, logits_val, _ = model.forward_inference(val_x, val_edge_index)
            
            # Compute loss
            loss_val, _ = model.compute_loss(
                val_x, recon_val, logits_val, val_y, val_edge_index, z_val
            )
            val_loss += loss_val.item()
            
            # Accuracy
            pred_classes = logits_val.argmax(dim=1)
            correct += (pred_classes == val_y).sum().item()
            total += val_y.size(0)
    
    avg_val_loss = val_loss / len(val_loader)
    val_acc = correct / total
    
    history['val_loss'].append(avg_val_loss)
    history['val_acc'].append(val_acc)
    
    # Learning rate scheduling
    scheduler.step(val_acc)
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_epoch = epoch
        torch.save(model.state_dict(), 'best_gae_model.pth')
        save_indicator = " 💾 SAVED"
    else:
        save_indicator = ""
    
    # Print progress
    if epoch % 1 == 0:
        print(f"Epoch {epoch:3d}/{CONFIG['num_epochs']} | "
              f"Recon: {epoch_losses['recon']:.4f} | "
              f"Class: {epoch_losses['class']:.4f} | "
              f"Graph: {epoch_losses['graph']:.4f} | "
              f"Val Loss: {avg_val_loss:.4f} | "
              f"Val Acc: {val_acc:.4f}{save_indicator}")

# ============================================================
# 5.3: Training Summary
# ============================================================

print(f"\n{'='*70}")
print("✅ TRAINING COMPLETE")
print(f"{'='*70}")
print(f"\n🏆 Best Results:")
print(f"   Epoch: {best_epoch}/{CONFIG['num_epochs']}")
print(f"   Validation Accuracy: {best_val_acc:.4f} ({best_val_acc*100:.2f}%)")
print(f"   Model saved: best_gae_model.pth")

## Section 6️⃣: Evaluate and Visualize Results

Extract latent representations, visualize with t-SNE, and compute comprehensive metrics.

In [ ]:
# ============================================================
# SECTION 6: EVALUATE AND VISUALIZE
# ============================================================

print("\n" + "="*70)
print("EVALUATION AND VISUALIZATION")
print("="*70)

# ============================================================
# 6.1: Load Best Model and Extract Predictions
# ============================================================

print("\n📊 Loading best model and extracting predictions...")

# Load best model
model.load_state_dict(torch.load('best_gae_model.pth', map_location=CONFIG['device']))
model.eval()

# Collect all predictions and representations
all_latents = []
all_predictions = []
all_true_labels = []
all_reconstructed = []
all_original = []

with torch.no_grad():
    for val_x, val_y in val_loader:
        val_x = val_x.to(CONFIG['device'])
        
        # Build graph
        val_features = model.ecg_encoder(val_x)
        val_edge_index = build_knn_graph(val_features, k=CONFIG['k_neighbors'])
        
        # Inference
        z_val, recon_val, logits_val, pred_classes = model.forward_inference(val_x, val_edge_index)
        
        # Collect data
        all_latents.append(z_val.cpu().numpy())
        all_predictions.append(pred_classes.cpu().numpy())
        all_true_labels.append(val_y.cpu().numpy())
        all_reconstructed.append(recon_val.cpu().numpy())
        all_original.append(val_x.cpu().numpy())

# Concatenate
latent_np = np.concatenate(all_latents, axis=0)
predictions_np = np.concatenate(all_predictions, axis=0)
true_labels_np = np.concatenate(all_true_labels, axis=0)
reconstructed_np = np.concatenate(all_reconstructed, axis=0)
original_np = np.concatenate(all_original, axis=0)

print(f"✅ Predictions collected")
print(f"   Latent representations: {latent_np.shape}")
print(f"   Total validation samples: {len(predictions_np):,}")

# ============================================================
# 6.2: Plot Training History
# ============================================================

print("\n📈 Plotting training history...")

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Loss curves
axes[0].plot(history['train_loss'], label='Total Train Loss', linewidth=2.5, color='steelblue')
axes[0].plot(history['val_loss'], label='Total Val Loss', linewidth=2.5, color='darkorange')
axes[0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Loss', fontsize=12, fontweight='bold')
axes[0].set_title('Training & Validation Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].set_yscale('log')

# Accuracy curves
axes[1].plot(history['val_acc'], label='Validation Accuracy', linewidth=2.5, color='seagreen')
axes[1].axhline(best_val_acc, color='red', linestyle='--', linewidth=2, 
                label=f'Best: {best_val_acc:.4f} (Epoch {best_epoch})')
axes[1].fill_between(range(len(history['val_acc'])), history['val_acc'], alpha=0.3, color='seagreen')
axes[1].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
axes[1].set_title('Classification Accuracy', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([0, 1.05])

plt.suptitle(f'Training Progress (Best Val Acc: {best_val_acc:.2%})', 
             fontsize=15, fontweight='bold')
plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Training history saved as: training_history.png")

In [ ]:
# ============================================================
# 6.3: Compute Classification Metrics
# ============================================================

print("\n" + "="*70)
print("CLASSIFICATION METRICS")
print("="*70)

# Overall accuracy
overall_accuracy = accuracy_score(true_labels_np, predictions_np)
macro_f1 = f1_score(true_labels_np, predictions_np, average='macro')
weighted_f1 = f1_score(true_labels_np, predictions_np, average='weighted')

print(f"\n✅ Overall Performance:")
print(f"   Accuracy:      {overall_accuracy:.4f} ({overall_accuracy*100:.2f}%)")
print(f"   Macro F1:      {macro_f1:.4f}")
print(f"   Weighted F1:   {weighted_f1:.4f}")

# Per-class metrics
print(f"\n✅ Per-Class Performance:")
print(classification_report(true_labels_np, predictions_np, 
                          target_names=CLASS_NAMES, digits=4))

# Confusion matrix
conf_matrix = confusion_matrix(true_labels_np, predictions_np)

# ============================================================
# 6.4: Visualize with t-SNE
# ============================================================

print("\n🔍 Computing t-SNE visualization...")

tsne = TSNE(n_components=2, random_state=SEED, perplexity=30, n_iter=1000)
latent_2d = tsne.fit_transform(latent_np)

print("✅ t-SNE computed")

# Plot t-SNE with predictions
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

colors = ['#2ecc71', '#e74c3c', '#3498db', '#f39c12', '#9b59b6']

# True classes
ax = axes[0]
for class_id in range(5):
    mask = true_labels_np == class_id
    ax.scatter(latent_2d[mask, 0], latent_2d[mask, 1],
              s=50, alpha=0.7, c=colors[class_id],
              label=CLASS_NAMES[class_id],
              edgecolors='black', linewidth=0.5)

ax.set_title('t-SNE: Ground Truth Classes', fontsize=15, fontweight='bold')
ax.set_xlabel('t-SNE Component 1', fontsize=12, fontweight='bold')
ax.set_ylabel('t-SNE Component 2', fontsize=12, fontweight='bold')
ax.legend(loc='best', framealpha=0.95, fontsize=10)
ax.grid(True, alpha=0.3)

# Predicted classes
ax = axes[1]
for class_id in range(5):
    mask = predictions_np == class_id
    ax.scatter(latent_2d[mask, 0], latent_2d[mask, 1],
              s=50, alpha=0.7, c=colors[class_id],
              label=CLASS_NAMES[class_id],
              edgecolors='black', linewidth=0.5)

ax.set_title(f't-SNE: Predicted Classes (Acc: {overall_accuracy:.1%})', 
            fontsize=15, fontweight='bold')
ax.set_xlabel('t-SNE Component 1', fontsize=12, fontweight='bold')
ax.set_ylabel('t-SNE Component 2', fontsize=12, fontweight='bold')
ax.legend(loc='best', framealpha=0.95, fontsize=10)
ax.grid(True, alpha=0.3)

plt.suptitle('Latent Space Visualization: True vs Predicted', 
            fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('tsne_visualization.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ t-SNE visualization saved as: tsne_visualization.png")

In [ ]:
# ============================================================
# 6.5: Comprehensive Performance Report
# ============================================================

print("\n" + "="*70)
print("COMPREHENSIVE PERFORMANCE REPORT")
print("="*70)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Confusion Matrix
ax = axes[0, 0]
conf_matrix_norm = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
sns.heatmap(conf_matrix_norm, annot=True, fmt='.3f', cmap='RdYlGn',
           xticklabels=[name.split(' (')[0] for name in CLASS_NAMES],
           yticklabels=[name.split(' (')[0] for name in CLASS_NAMES],
           ax=ax, vmin=0, vmax=1, cbar_kws={'label': 'Proportion'})
ax.set_title('Confusion Matrix (Normalized)', fontsize=14, fontweight='bold')
ax.set_xlabel('Predicted', fontsize=12, fontweight='bold')
ax.set_ylabel('True', fontsize=12, fontweight='bold')

# Plot 2: Per-Class Accuracy
ax = axes[0, 1]
class_accuracies = []
for class_id in range(5):
    mask = true_labels_np == class_id
    if mask.sum() > 0:
        acc = (predictions_np[mask] == class_id).mean()
        class_accuracies.append(acc)
    else:
        class_accuracies.append(0)

bars = ax.bar(range(5), class_accuracies, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax.axhline(overall_accuracy, color='red', linestyle='--', linewidth=2, label=f'Overall: {overall_accuracy:.3f}')
ax.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax.set_title('Per-Class Accuracy', fontsize=14, fontweight='bold')
ax.set_xticks(range(5))
ax.set_xticklabels([name.split(' (')[0] for name in CLASS_NAMES], rotation=45, ha='right')
ax.set_ylim([0, 1.1])
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

# Plot 3: Loss Component Breakdown
ax = axes[1, 0]
x = np.arange(len(history['train_loss']))
ax.plot(x, history['train_recon_loss'], label='Reconstruction', linewidth=2, color='#3498db')
ax.plot(x, history['train_class_loss'], label='Classification', linewidth=2, color='#e74c3c')
ax.plot(x, history['train_graph_loss'], label='Graph', linewidth=2, color='#2ecc71')
ax.set_xlabel('Epoch', fontsize=12, fontweight='bold')
ax.set_ylabel('Loss', fontsize=12, fontweight='bold')
ax.set_title('Loss Components Over Time', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Plot 4: Summary Statistics
ax = axes[1, 1]
ax.axis('off')

summary_text = f"""
MODEL PERFORMANCE SUMMARY
{'='*40}

Overall Metrics:
  • Accuracy:           {overall_accuracy:.4f} ({overall_accuracy*100:.2f}%)
  • Macro F1-Score:     {macro_f1:.4f}
  • Weighted F1-Score:  {weighted_f1:.4f}

Training Details:
  • Best Epoch:         {best_epoch}/{CONFIG['num_epochs']}
  • Best Val Accuracy:  {best_val_acc:.4f}
  • Total Samples:      {len(true_labels_np):,}
  • Correct Preds:      {(predictions_np == true_labels_np).sum():,}
  • Incorrect Preds:    {(predictions_np != true_labels_np).sum():,}

Model Architecture:
  • Total Parameters:   {sum(p.numel() for p in model.parameters()):,}
  • Latent Dimension:   {CONFIG['latent_dim']}
  • k-NN Neighbors:     {CONFIG['k_neighbors']}
  • Device:             {CONFIG['device']}
"""

ax.text(0.1, 0.95, summary_text, transform=ax.transAxes,
       fontsize=11, verticalalignment='top', family='monospace',
       bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.suptitle('Comprehensive Model Evaluation', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('model_evaluation.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Evaluation report saved as: model_evaluation.png")

print(f"\n{'='*70}")


### 📚 Next Steps

**To use this for your own data:**

1. Update `CONFIG['train_path']` with your CSV file path
2. Adjust class names if needed
3. Run all cells again

**To deploy in production:**

```python
# Use src modules in FastAPI
from src.models import EnhancedJointECGModel
from src.inference import ECGPredictor

# Or use via CLI
python src/train.py --data data.csv --epochs 200
python src/inference.py --model best_model.pth --input test.csv
```
